In [1]:
#Importing necessary libraries and dependencies
import torch
import math
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras
import pandas as pd #for loading data
import os
import PIL
import cv2
from random import uniform

from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
import keras.layers
from keras.layers import MaxPool2D, AveragePooling2D, Flatten, Dense, Lambda, LayerNormalization, Dropout
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam, SGD
from torchvision import datasets, transforms
from PIL import Image

In [2]:
#Import and Load data

#Batch and Test size
batch_size = 75
test_size = 300
height = 75
width = 320

#Read csv file 7971
csv_directory = r'insert directory name here'

opened_data = pd.read_csv(csv_directory)

#Define X and Y data
X_data = opened_data[['Center', 'Left', 'Right']].values
Y_data = np.ndarray.round(opened_data['steering'].values, 3)

#Define train/test split and batch size
train_x, valid_x, train_y, valid_y = train_test_split(X_data, Y_data, test_size = test_size, random_state = 0)

print(train_x.shape)

(15492, 3)


In [3]:
#Batch generation function

def read_image(directory, file):
    return cv2.imread(os.path.join(directory, file.strip()))

def crop(image):
    return image [60:-25, :, :]

def yuv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

def choose(directory, center, left, right, steering_angle):
    choice = np.random.choice(3)
    if choice == 0:
        return read_image(directory, left), steering_angle + 0.2
    elif choice == 1:
        return read_image(directory, right), steering_angle -0.2
    return read_image(directory, center), steering_angle

def flip(image, steering_angle):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
    return image, steering_angle

def augment(directory, center, left, right, steering):
    image, steering = choose(directory, center, left, right, steering)
    image, steering = flip(image, steering)
    return image, steering

def resize (image):
    return cv2.resize(image, (width, height), cv2.INTER_AREA)

def generate_batches(directory, x_data, y_data, size):

    #Set array sizes
    image_arr = np.empty([size, 75, 320, 3])
    steering_arr = np.empty(size)

    while True:
        i = 0
        for position in np.random.permutation(x_data.shape[0]):
            center, left, right = x_data[position]
            steering = y_data[position]
            #image, steering = augment(directory, center, left, right, steering)
            image = read_image(directory, center)
            #image = yuv(image)
            image_arr[i] = crop(image)
            steering_arr[i] = steering

            i+= 1
            if i == batch_size:
                break
        yield image_arr, steering_arr


In [4]:
image_array = generate_batches(csv_directory, train_x, train_y, batch_size)
print(next(iter(image_array)))

(array([[[[ 83., 173., 204.],
         [ 87., 177., 208.],
         [ 93., 183., 214.],
         ...,
         [ 45., 112., 127.],
         [ 43., 109., 127.],
         [ 41., 107., 125.]],

        [[ 91., 180., 214.],
         [ 90., 179., 213.],
         [ 89., 178., 212.],
         ...,
         [ 42., 104., 120.],
         [ 38., 100., 118.],
         [ 35.,  97., 115.]],

        [[ 92., 179., 211.],
         [ 89., 176., 208.],
         [ 84., 171., 203.],
         ...,
         [ 46., 102., 121.],
         [ 40.,  98., 117.],
         [ 40.,  98., 117.]],

        ...,

        [[117., 126., 123.],
         [103., 112., 109.],
         [113., 122., 119.],
         ...,
         [ 95., 102.,  95.],
         [117., 124., 117.],
         [140., 147., 140.]],

        [[115., 124., 121.],
         [118., 127., 124.],
         [134., 143., 140.],
         ...,
         [156., 163., 156.],
         [141., 148., 141.],
         [121., 128., 121.]],

        [[111., 120., 117.],
      

In [5]:
#Set a checkpoint

checkpoint = ModelCheckpoint('model={epoch:03d}.h5',
             monitor = 'loss',
             verbose = 0,
             save_best_only = True,
             mode = 'auto')


In [7]:
#Define model architecture
seq_model = Sequential()

# seq_model.add(Conv2D(24, 5, strides = 2, activation = 'elu', input_shape = [75, 320, 3]))
# seq_model.add(Lambda(lambda x: x/127.5-1.0))
# #seq_model.add(Dropout(0.2))
# #seq_model.add(Conv2D(64, 3, strides = 2, activation = 'elu'))
# seq_model.add(Flatten())
# seq_model.add(Dense(1, activation = 'selu'))
# seq_model.summary()

seq_model = Sequential()
seq_model.add(Lambda(lambda x: x/127.5-1.0, input_shape = [75, 320, 3]))
seq_model.add(Conv2D(12, 15, strides = 2, activation = 'elu'))
seq_model.add(Conv2D(15, 18, strides = 2, activation = 'elu'))
#seq_model.add(Conv2D(18, 24, strides = 2, activation = 'elu'))
#seq_model.add(Dropout(0.2))
seq_model.add(Flatten())
#seq_model.add(Dense(11532, activation = 'elu'))
#seq_model.add(Dense(7611, activation = 'elu'))
seq_model.add(Dense(1, activation = 'elu'))
seq_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 75, 320, 3)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 153, 12)       8112      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 68, 15)         58335     
_________________________________________________________________
flatten (Flatten)            (None, 7140)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 7141      
Total params: 73,588
Trainable params: 73,588
Non-trainable params: 0
_________________________________________________________________


In [8]:
#Set optimizer and criterion

seq_model.compile(optimizer = SGD(lr = 0.0001), loss = tf.keras.losses.MeanSquaredError(), metrics = ['accuracy'] )

In [ ]:
#Load an existing model to train
seq_model.load_weights(r'insert directory name here')

In [ ]:
#Train the network
n_samples = 900
n_epochs = 10
validation_sample_length = 50

history = seq_model.fit_generator(generate_batches(csv_directory, train_x, train_y, batch_size), 
                        steps_per_epoch = n_samples,
                        epochs = n_epochs,
                        max_queue_size=10,
                        validation_data = generate_batches(csv_directory, valid_x, valid_y, batch_size),
                        validation_steps = validation_sample_length,
                        callbacks = [checkpoint],
                        verbose = 1,
                        shuffle = True)

In [ ]:
seq_model.save('name of net.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss of Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training loss', 'validation loss'], loc = 'upper left')
plt.show()
plt.savefig('graph3.1359.jpg')